In [16]:
import matplotlib.pyplot as plt
import pandas as pd
#import geopandas as gpd
from census import Census
from us import states
from pathlib import Path
import geoviews as gv
import geoviews.feature as gf
import os


In [17]:
# Import U.S. Census API Key
from config import api_key

# Create an instance of the Census library
c = Census(
    api_key,
    year = 2020
)

In [14]:
# all_counties = []
# #Loop through all the states to get county data
# for state in states.STATES:
#     county_data = c.acs5.state_county(('NAME', 'B19013_001E', 'B01003_001E'), state.fips,  Census.ALL)
#     all_counties.append(pd.DataFrame(county_data))

In [51]:
# #Concatenate the county data for all states into a single DataFrame
# all_counties_df = pd.concat(all_counties)
# all_counties_df

In [50]:
# all_counties_df = all_counties_df.rename(columns={
#     'NAME': 'county',
#     'B19013_001E': 'median_household_income',
#     'B01003_001E': 'total_population'
# })

# all_counties_df

In [14]:
all_counties = []
for state in states.STATES:
    state_fips = state.fips
    county_data = c.acs5.state_county(('NAME', 'B19013_001E', 'B01003_001E'), state_fips, '*', year=2020)
    all_counties.append(pd.DataFrame(county_data))

#  Concatenate the county data for all states into a single DataFrame
all_counties_df = pd.concat(all_counties)

#  Rename the columns to make them more readable
all_counties_df = all_counties_df.rename(columns={
    'NAME': 'County',
    'B19013_001E': 'median_household_income',
    'B01003_001E': 'total_population'
})

#  Reset the index
all_counties_df = all_counties_df.reset_index(drop=True)

#  Print the DataFrame
all_counties_df.head()

,County,median_household_income,total_population,state,county
0,"Autauga County, Alabama",57982.0,55639.0,01,001
1,"Baldwin County, Alabama",61756.0,218289.0,01,003
2,"Barbour County, Alabama",34990.0,25026.0,01,005
3,"Bibb County, Alabama",51721.0,22374.0,01,007
4,"Blount County, Alabama",48922.0,57755.0,01,009


In [15]:
# Convert the county column to string type
all_counties_df['County'] = all_counties_df['County'].astype(str)
# Split the county and state in the rows of the county column
all_counties_df[['County', 'state']] = all_counties_df['County'].str.split(', ', expand=True)
all_counties_df= all_counties_df.rename(columns={"state": "State"})
all_counties_df= all_counties_df.astype(str)

all_counties_df


,County,median_household_income,total_population,State,county
0,Autauga County,57982.0,55639.0,Alabama,001
1,Baldwin County,61756.0,218289.0,Alabama,003
2,Barbour County,34990.0,25026.0,Alabama,005
3,Bibb County,51721.0,22374.0,Alabama,007
4,Blount County,48922.0,57755.0,Alabama,009
...,...,...,...,...,...
3137,Hot Springs County,53398.0,4550.0,Wyoming,017
3138,Natrona County,62168.0,80067.0,Wyoming,025
3139,Platte County,57784.0,8572.0,Wyoming,031
3140,Sweetwater County,73384.0,43352.0,Wyoming,037


In [11]:
# Read data from csv file
file = Path("Resources/Diabetes - 2020.csv")

diabetes_df = pd.read_csv(file)

diabetes_df = pd.DataFrame(diabetes_df[["State", "County", "Number"]])

diabetes_df = diabetes_df.rename(columns={"Number": "Diabetes Estimate"})

# remove rows with text in Diabetes Estimate column

diabetes_df = diabetes_df[diabetes_df["Diabetes Estimate"] != "Suppressed"]
diabetes_df = diabetes_df[diabetes_df["Diabetes Estimate"] != "No Data"]
diabetes_df["Diabetes Estimate"] = diabetes_df["Diabetes Estimate"].astype(str)
# reset index
diabetes_df = diabetes_df.reset_index(drop=True)
diabetes_df

,State,County,Diabetes Estimate
0,Hawaii,Kalawao County,12
1,Texas,King County,17.6
2,Texas,Kenedy County,26.5
3,Nebraska,Arthur County,29.3
4,Nebraska,Blaine County,33.1
...,...,...,...
3135,California,Orange County,229477.5
3136,Arizona,Maricopa County,316604.6
3137,Texas,Harris County,344810.7
3138,Illinois,Cook County,356695.4


In [12]:
merged_df = pd.merge(diabetes_df, all_counties_df, on=['State', 'County'], how='right')
merged_df


,State,County,Diabetes Estimate,median_household_income,total_population,county
0,Alabama,Autauga County,3861.8,57982.0,55639.0,001
1,Alabama,Baldwin County,19353.8,61756.0,218289.0,003
2,Alabama,Barbour County,2200.8,34990.0,25026.0,005
3,Alabama,Bibb County,1708.3,51721.0,22374.0,007
4,Alabama,Blount County,5043.5,48922.0,57755.0,009
...,...,...,...,...,...,...
3137,Wyoming,Hot Springs County,357.7,53398.0,4550.0,017
3138,Wyoming,Natrona County,4726.8,62168.0,80067.0,025
3139,Wyoming,Platte County,595.1,57784.0,8572.0,031
3140,Wyoming,Sweetwater County,2368.3,73384.0,43352.0,037


In [7]:
print(merged_df.dtypes)

State                      object
County                     object
Diabetes Estimate          object
median_household_income    object
total_population           object
county                     object
dtype: object
